### Sky Position

Given an event timestamp (from a dat file) and beam number, compute the (RA, Dec), (l, b), (alt, az) of the beam centre and a list of pulsars within a 2 degree radius of the beam.

In [1]:
import numpy as np
import pandas as pd
import astropy.coordinates
import astropy.time
from astropy import units as u

In [2]:
# fixed beam declinations
beamDecs = np.array([7.9, 11., 13.3, 15.85, 19., 22., 24.9, 28.66]) # beams 1-8

# LOFAR-UK HBA (X, Y, Z)
loc = astropy.coordinates.EarthLocation(4008461.905110000, -100376.559511000, 4943716.904, unit='m')
print(loc.lat, loc.lon)

51d08m36.7694s -1d26m04.032s


### ATNF pulsar catalogue

The ATNF pulsar catalogue contains details of all published pulsars.

http://www.atnf.csiro.au/people/pulsar/psrcat/

```
psrcat -nonumber -nohead -o short -c 'JNAME RAJD DECJD P0 P1 W50 S1400 DM SPINDX SI414' -l "(W50>0)&&(P1>0)&&(S1400>0)" > 2018psrcat.csv
```

In [3]:
# psrcat
psrcatFile = '2018psrcat.csv'
df = pd.read_csv(psrcatFile, sep='\s+', header=None, names=['JNAME', 'RAJD', 'DECJD', 'P0', 'P1', 'W50',\
                                                            'S1400', 'DM', 'SPINDX', 'SI414'], na_values='*')
#print(df)
dfShort = df.drop(labels=['P1', 'SPINDX', 'SI414'], axis=1)

In [4]:
# example: Beam1_dm_D20160320T230545_block9
eventName = 'Beam1_dm_D20160320T230545_block9'

beamId = int(eventName[4])

dtStr = '%s-%s-%s %s:%s:%s'%(eventName[10:14], eventName[14:16], eventName[16:18], eventName[19:21],\
                             eventName[21:23], eventName[23:25])

dec = astropy.coordinates.Angle(beamDecs[beamId - 1], u.degree)

t = astropy.time.Time(dtStr, scale='utc', location=loc)

lstGreenwich = t.sidereal_time('apparent', 'greenwich') # beams are pointed slightly east of the local meridian
lst = t.sidereal_time('apparent')
ra = lstGreenwich

c = astropy.coordinates.SkyCoord(ra=ra, dec=dec, frame='icrs')
print(c)
print(c.galactic)
print(c.transform_to(astropy.coordinates.AltAz(obstime=t, location=loc)))

<SkyCoord (ICRS): (ra, dec) in deg
    (165.34230978, 7.9)>
<SkyCoord (Galactic): (l, b) in deg
    (244.23140747, 57.61706051)>
<SkyCoord (AltAz: obstime=2016-03-20 23:05:45.000, location=(4008461.90511, -100376.559511, 4943716.904) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (177.6190521, 46.64515403)>


In [5]:
# find pulsars in the beam
beamRadius = 2.0 # degrees

print(dfShort[np.sqrt((df['RAJD'] - ra.degree)**2. + (df['DECJD'] - dec.degree)**2.) < beamRadius])

Empty DataFrame
Columns: [JNAME, RAJD, DECJD, P0, W50, S1400, DM]
Index: []
